# Lecture 3: Finite Differences

In [1]:
# Initialisation

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from scipy.sparse import diags
from ipywidgets import *
from IPython.display import display, Markdown

x   = Symbol('x')
dxs = Symbol('\Delta x')

In the lecture, we examined finite difference methods for differential equations. In this notebook, we will graphically examine these finite differences methods. Firstly, the forward, backward and centred finite difference approximations for first-order derivatives are as such

Forward difference: $\frac{f(x+\Delta x)-f(x)}{\Delta x}$

Backward difference: $\frac{f(x)-f(x-\Delta x)}{\Delta x}$

Centred difference: $\frac{f(x+\Delta x)-f(x-\Delta x))}{2\Delta x}$

In this example, we will examine the first derivative of $sin(x)$. Analytically, we know the solution to be $cos(x)$. We will look at how each difference method approximates the first derivative. 

The variables in the cell below can be subsequently changed to investigate different equations.

In [2]:
## Edit variables here
# Equation to be differentiated
f = sin(x)

# Define start and end of range for plot
x_start = 0
x_end   = np.pi
x_space = 100

We plot both the analytical solution, $cos(x)$, and the three difference equations.

In [3]:
## Code to plot graphs
# Create Derivative object
dfdx = Derivative(f,x)

# Create finite difference equations
fd = (f.subs(x,UnevaluatedExpr(x+dxs))-f)/dxs
bd = (f-f.subs(x,UnevaluatedExpr(x-dxs)))/dxs
cd = (f.subs(x,UnevaluatedExpr(x+dxs))-f.subs(x,UnevaluatedExpr(x-dxs)))/(2*dxs)

# Display finite difference equations
display(Markdown('To find $' + latex(dfdx) + '$ :'))
display(Markdown('The analytical solution is $' + latex(dfdx.doit()) + '$ .'))
display(Markdown('The forward difference equation is $' + latex(fd) + '$ .'))
display(Markdown('The backward difference equation is $' + latex(bd) + '$ .'))
display(Markdown('The centred difference equation is $' + latex(cd) + '$ .'))

# Calculate values
x_val = np.linspace(x_start, x_end, x_space)

# Plot analytical solution
def update1(dx=0.5):
    dyval = lambdify(x, dfdx.doit(), modules=['numpy'])(x_val)
    fdval = lambdify(x, fd.subs(dxs,dx), modules=['numpy'])(x_val)
    bdval = lambdify(x, bd.subs(dxs,dx), modules=['numpy'])(x_val)
    cdval = lambdify(x, cd.subs(dxs,dx), modules=['numpy'])(x_val)
    
    plt.plot(x_val, dyval, color='tab:blue')
    plt.plot(x_val, fdval, color='tab:orange')
    plt.plot(x_val, bdval, color='tab:green')
    plt.plot(x_val, cdval, color='tab:red')
    
    plt.grid()
    plt.legend(['Analytical Solution', 'Forward Difference', 'Backward Difference', 'Centred Difference'])

# Slider
interact1 = interact(update1, dx=FloatLogSlider(min=-2, max=0.5, value=0.5, continuous_update=False, description='$\Delta x$'))

To find $\frac{d}{d x} \sin{\left(x \right)}$ :

The analytical solution is $\cos{\left(x \right)}$ .

The forward difference equation is $\frac{- \sin{\left(x \right)} + \sin{\left(\Delta x + x \right)}}{\Delta x}$ .

The backward difference equation is $\frac{\sin{\left(x \right)} - \sin{\left(- \Delta x + x \right)}}{\Delta x}$ .

The centred difference equation is $\frac{- \sin{\left(- \Delta x + x \right)} + \sin{\left(\Delta x + x \right)}}{2 \Delta x}$ .

interactive(children=(FloatLogSlider(value=0.5, continuous_update=False, description='$\\Delta x$', max=0.5, m…

Moving the slider adjusts the value of $\Delta x$ used in the finite difference methods. By moving the slider to the left, $\Delta x$ is decreased, causing the finite difference methods to become more accurate when compared to the analytical solution.

## Errors

We now examine the truncation error $\varepsilon$ of each finite difference method. The truncation error is the difference between the exact value and the numerical approximation.

We plot again the analytical solution and the three difference equations. This time, we also find the points at which the maximum errors are found, which are annotated with double-headed arrows.

In [4]:
def update2(dx=0.5):
    dyval = lambdify(x, dfdx.doit(), modules=['numpy'])(x_val)
    fdval = lambdify(x, fd.subs(dxs,dx), modules=['numpy'])(x_val)
    bdval = lambdify(x, bd.subs(dxs,dx), modules=['numpy'])(x_val)
    cdval = lambdify(x, cd.subs(dxs,dx), modules=['numpy'])(x_val)
    
    plt.plot(x_val, dyval, color='tab:blue')
    plt.plot(x_val, fdval, color='tab:orange')
    plt.plot(x_val, bdval, color='tab:green')
    plt.plot(x_val, cdval, color='tab:red')
    
    plt.grid()
    plt.legend(['Analytical Solution', 'Forward Difference', 'Backward Difference', 'Centred Difference'])
    
    # Find maximum errors
    fe = np.amax(np.abs(dyval-fdval))
    be = np.amax(np.abs(dyval-bdval))
    ce = np.amax(np.abs(dyval-cdval))
    
    fa = np.argmax(np.abs(dyval-fdval))
    ba = np.argmax(np.abs(dyval-bdval))
    ca = np.argmax(np.abs(dyval-cdval))
    
    # Plot double-headed arrows
    plt.annotate(s='', xy=(x_val[fa],dyval[fa]), xytext=(x_val[fa],fdval[fa]), arrowprops=dict(arrowstyle='<->', shrinkA=0, shrinkB=0, color='tab:orange'))
    plt.annotate(s='', xy=(x_val[ba],dyval[ba]), xytext=(x_val[ba],bdval[ba]), arrowprops=dict(arrowstyle='<->', shrinkA=0, shrinkB=0, color='tab:green'))
    plt.annotate(s='', xy=(x_val[ca],dyval[ca]), xytext=(x_val[ca],cdval[ca]), arrowprops=dict(arrowstyle='<->', shrinkA=0, shrinkB=0, color='tab:red'))
    
    # Text output
    print(f'The maximum absolute error for  forward difference is %.3f' % fe)
    print(f'The maximum absolute error for backward difference is %.3f' % be)
    print(f'The maximum absolute error for  centred difference is %.3f' % ce)

# Slider
interact2 = interact(update2, dx=FloatLogSlider(min=-2, max=0.5, value=1.0, continuous_update=False, description='$\Delta x$'))

interactive(children=(FloatLogSlider(value=1.0, continuous_update=False, description='$\\Delta x$', max=0.5, m…

The absolute values of the maximum errors are consistent with the observation that both forward difference and backward difference are first order approximations, resulting in similar accuracies, while centred difference is a second order approximation, and thus is more accurate than both forward and backward difference.

## One-sided formula

It is common to use the centred difference method. However, this may not be always possible (e.g. boundary conditions such as impermeability at a wall). Hence, a one-sided finite difference scheme can be derived instead, by considering a stencil only consisting of stencil points on one side. We are also able to develop higher order approximations by using more stencil points, or carefully selecting stencil points, to eliminate the lowest-ordered term of the truncation error from the Taylor expansions.

In this example we examine how coefficients can be calculated given a stencil of selected points on one side. In this case, we are looking at forward differences.

For equispaced stencil points $x_0, x_1, ..., x_p$, we consider coefficients $a_0, a_1, ..., a_p$. By then considering the relevant equations, we derive the system of linear equations for the $m$-th derivative as such:

$$
\begin{bmatrix}
1 & 1 & ... & 1 & 1 \\\\
0 & 1 & ... & (p-1) & p \\\\
0 & 1 & ... & (p-1)^2 & p^2 \\\\
... & ... & ... & ... & ... \\\\
0 & 1 & ... & (p-1)^p & p^p
\end{bmatrix}
\begin{bmatrix}
a_0 \\\\ a_1 \\\\ a_2 \\\\  ... \\\\ a_p
\end{bmatrix}
=
m!
\begin{bmatrix}
\delta_{0,m} \\\\ \delta_{1,m} \\\\ \delta_{2,m} \\\\ ... \\\\ \delta_{p,m}
\end{bmatrix}
$$

where $\delta_{i,j}$ is the Kronecker delta.

Below, we can select the derivative we want to find the coefficients for and the points we want for our stencil.

In [5]:
def checkboxClick(d=1,x0=True,x1=True,x2=False,x3=False,x4=False,x5=False):
    l = [x0,x1,x2,x3,x4,x5]
    ll = ['x_0','x_1','x_2','x_3','x_4','x_5']
    i = l.count(True)
        
    if i <= d:
        display(Markdown('Insufficient stencil points'))
        return
    
    p = [z for z in range(6) if l[z] is True]
    ll = [ll[z] for z in p]
    s = np.array([[j**k for j in p] for k in p])
    r = np.array([np.math.factorial(d) if d==z else 0 for z in range(i)])
    
    a = Matrix(list(linsolve((Matrix(s),Matrix(r)),[x]))[0])
    
    display(Markdown('To solve: $$'+latex(Matrix(s))+'^{-1}'+latex(Matrix(r))+'='+latex(a)+'$$'))
    display(Markdown('The coefficients are: '))
    for lli in range(len(ll)):
        display(Markdown('$'+latex(ll[lli])+'$ : $'+latex(a[lli])+"$"))
    display(Markdown('The order of accuracy is '+latex(i-d)+'.'))
        
    
# Create checkboxes
display(Markdown('Select derivative and stencil points:'))
interact3 = interact(checkboxClick,
                     d =IntSlider(min=1,max=5,description='Derivative'),
                     x0=Checkbox(description='$x_0$', value=True),
                     x1=Checkbox(description='$x_1$', value=True),
                     x2=Checkbox(description='$x_2$'),
                     x3=Checkbox(description='$x_3$'),
                     x4=Checkbox(description='$x_4$'),
                     x5=Checkbox(description='$x_5$'),
                    )

Select derivative and stencil points:

interactive(children=(IntSlider(value=1, description='Derivative', max=5, min=1), Checkbox(value=True, descrip…

## Finite difference solution of elliptic equation

Consider the elliptic equation

$$
u_{xx} = f(x) = -\frac{w(x)}{T}
$$

with boundary conditions

$$
x \in [-1, 1], \quad u(-1) = u(1) = 0
$$

Recalling our matrix system $Au = f^*$ using centred approximation: 

$$
\left [  \begin{array}{ccccc}
-2 & 1 & & & \\\\ 1 & -2 & 1 & &\\\\ & \ddots & \ddots & \ddots &
 \\\\ & & 1 & -2 & 1 \\\\ & & & 1 & -2 \end{array} \right ] \left
[ \begin{array}{c} u_1 \\\\ u_2 \\\\ \vdots \\\\ u_{N-2} \\\\ u_{N-1} \end{array} \right ]
 =\left [ \begin{array}{l}\Delta x^2 f_1 - \alpha\\\\ \Delta x^2 f_2 \\\\ \hspace{5mm} \vdots \\\\  \Delta x^2 f_{N-2}\\\\ \Delta x^2 f_{N-1}- \beta \end{array} \right ].
$$

We can now calculate the approximate $u(x)$ values.

In [6]:
# Edit w(x) and T
w = cos(x*np.pi/2)
T = 0.2
pl = list(np.linspace(-1,1,101))

# Plotting function
def update4(N=11):
    dx= 2/(N-1)
    r = list(np.linspace(-1,1,num=N))
    wl= Matrix([w.subs(x,z) for z in r[1:-1]])
    A = diags([1,-2,1],[-1,0,1],shape=(N-2,N-2)).toarray()
    b = (np.array((dx**2)*(wl/T))).astype(np.float64) # flip sign of f(x) for plotting
    u = np.linalg.solve(A,b)
    
    un = np.array(u.tolist()).astype(np.float64)
    un = np.insert(un,(0,N-2),0)
    
    # Analytical solution
    w2x =  (1/T)*integrate(integrate(w, x))
    c1  =  (1/2)*(w2x.subs(x,-1)-w2x.subs(x,1))
    c2  = -(1/2)*(w2x.subs(x,-1)+w2x.subs(x,1))
    ua  = w2x+c1*x+c2
    ual = np.array([ua.subs(x,z) for z in pl])
    
    # Plotting
    plt.plot(pl, np.array([w.subs(x,z) for z in pl]))
    plt.plot(pl, ual)
    plt.plot(r, un)
    plt.grid()
    plt.legend(['$w(x)$', 'Analytical $u(x)$', 'Finite Difference $u(x)$'])
    
# Slider
interact4 = interact(update4, N=IntSlider(min=3,max=11,value=3, continuous_update=False))

interactive(children=(IntSlider(value=3, continuous_update=False, description='N', max=11, min=3), Output()), …

The slider $N$ can be shifted to adjust the number of points taken.

We can then compare our finite difference approximation to the analytical solution, which can be trivially found.